   # ------------------------- Estudios de Punto de Pedido --------------------------------------------

Se importan las diferentes bibliotecas que se van a emplear.  
Se configuran las salidas y la precisión de los datos.  
Se fija t en 365 días.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 200, "display.max_columns",200)
np.set_printoptions(precision=1, suppress=True)

In [2]:
t=364

# Primera parte

En la primera parte se realiza la lectura de los datos y pequeñas transformaciones para su posterior uso.  
Las entradas son las compras y los envíos recibidos procedentes de otras bases  
Nombre de las columnas:  
0 : producto (PN)  
2 : punto de pedido fijado por la empresa, tambien llamado stock mínimo (SM)   
3 : cantidades compradas (en este caso)  
5 : precio por unidad  
5555: tiempo de reaprovisionamiento  

In [3]:
Compras=pd.read_csv('ENTRADAS.ALMACEN.csv').rename(columns={'0':0, '2':2, '1':1, '3':3, '5':5, '5555':5555})

In [4]:
Consumos=pd.read_csv('SALIDAS.ALMACEN.csv').rename(columns={'0':0, '1':1, '3':3})

Las salidas son la suma de los consumos y envíos a otras bases  
0 : producto (PN)  
1 : fecha de consumo  
3 : cantidad consumida (en este caso)  
33 : consumo medio diario  
6 : punto de pedido  
66: punto de pedido, más stock de seguridad  
8: stock de seguridad (s)  

El consumo total por producto  
Añadimos una columna con el consumo medio diario de cada producto

In [5]:
Consumo_total=Consumos.loc[:,[0,3]].groupby([0]).sum().reset_index() 
Consumo_total[33]=Consumo_total[3]/t

Se extrae el número de veces que se ha realizado una compra

In [6]:
numero_compras=Compras.loc[:,[0,5555]].groupby([0]).count().reset_index()

Se guardan los indices para su posterior uso

In [7]:
numero_compras_indices=Compras.loc[:,[0,5555]].groupby([0]).count()

El tiempo medio que tarda la compra en llegar al almacén

In [8]:
tiempo_medio_compras=round(Compras.loc[:,[0,5555]].groupby([0]).mean().reset_index(), 2)

Se agrupan los datos de compras y consumos por producto

In [9]:
Consumos_compra=(Compras.loc[:,[0,2]].merge(Consumo_total, on=0)).merge(tiempo_medio_compras, on=0)

Se define una función para obtener el stock de seguridad  
Esta función extrae el día en el cual ha llegado almenos el 95% de las compras de cada producto

In [10]:
def get_safe(d):           
    s=np.sort(d)          
    p=int(0.95 * len(d))
    return s[p]

Se aplica el resultado de la función a los datos

In [11]:
Nivel_seguridad_95=Compras.loc[:,[0,5555]].groupby([0]).agg(get_safe).rename(columns={5555: 55}).reset_index() 

Se agrupan datos

In [12]:
s=Consumos_compra.merge(Nivel_seguridad_95, on=0)

El stock de seguridad, es el resultado de multiplicar el día en el que ha llegado  
El 95% de las compras de un producto por el consumo medio anualde ese mismo producto.

In [13]:
s[8]=s[55]*s[33] 

Se extrae el nivel de seguridad de cada producto y se eliminan dublicados

In [14]:
s=s.loc[:,[0,8]]
s=s.drop_duplicates(subset=None,keep='first',inplace=False)

Se incorpora el precio medio de cada productos para hacer los calculos posteriores  
Se elimnan dublicados  
Se eliminan los NAs

In [15]:
Consumos_compra=Consumos_compra.merge(round(Compras.loc[:,[0,5]].groupby([0]).mean().reset_index(),2),on=0)
Consumos_compra=Consumos_compra.drop_duplicates(subset=None,keep='first',inplace=False)
Consumos_compra=Consumos_compra.dropna()

Se muestra información de los producto que se emplean como ejemplo

In [16]:
Consumos_compra[Consumos_compra[0].isin([1274,1620,2033])].loc[:,[0,2,3]].rename(columns={0:'PN',2:'SM',3:'Consumo anual'})

,PN,SM,Consumo anual
75,1274,432.0,1570
98,1620,42.0,150
115,2033,1.0,1


# Segunda parte

Tiempo que tarda una compra en llegar al almacén desde que se da la orden de compra  
Se extrae el número de veces que se ha realizado una compra  
Se convierte el numero de compras a array de numpy     

In [17]:
tiempo_llegada=Compras.loc[:,[0,5555]] 
numero_compras=Compras.loc[:,[0,5555]].groupby([0]).count()
numero_compras=np.array(numero_compras)

Se crea un dataframe donde los ptoductos  
son los indices y los dias del año son las columnas.  
Extraemos las colmunas, estas son los dias del año  
Posteriormente se convierten los datos a array de numpy    

In [18]:
frecuencia_llegada=pd.crosstab(index=tiempo_llegada[0], columns=tiempo_llegada[5555]) 
dias=np.array(frecuencia_llegada.columns)
frecuencia_llegada=np.array(frecuencia_llegada) 

La probabilidad de que llegue un pedido en un dia determinado

In [19]:
probabilidad_llegada=frecuencia_llegada/numero_compras 

Guardamos las probabilidades  
Fijamos los indices como nombre de los productos de tal forma que el indice coincida con el identificador del producto

In [20]:
probabilidad_llegada=pd.DataFrame(probabilidad_llegada, index=numero_compras_indices.index, columns=dias) 

Probabilidad del tiempo de llegada (días) de las compras

In [21]:
probabilidad_llegada.loc[[1274,1620,2033]].iloc[:,0:15] 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,,,,,,
1274,0.00,0.0,0.0,0.0,0.0,0.333333,0.333333,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.333333
1620,0.25,0.0,0.0,0.0,0.0,0.250000,0.000000,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.000000
2033,0.00,0.0,0.0,0.0,0.0,0.000000,0.500000,0.00,0.00,0.5,0.0,0.0,0.0,0.0,0.000000


El tiempo de llegada de cada producto por su probabilidad para obtener la media probabilistica  

In [22]:
probabilidad_llegada_dias=probabilidad_llegada*dias

Media probabilistica del tiempo de reaprovisionamiento

In [23]:
probabilidad_media_llegada=pd.DataFrame(probabilidad_llegada_dias.cumsum(axis=1)[probabilidad_llegada_dias.columns[-1]]).reset_index()  

Se parte de los datos comunes del apartado anterior

In [24]:
Consumos_compra_probabilidad=Consumos_compra.loc[:,[0,2,3,33,5]] 

Se añade el tiempo de llegadas de las compras obtenido con la esperanza matematica y se le asigna el mismo nombre  "5555"

In [25]:
Consumos_compra_probabilidad=Consumos_compra_probabilidad.merge(probabilidad_media_llegada, on=0).rename(columns={30:5555})

In [26]:
Consumos_compra_probabilidad[6]=Consumos_compra_probabilidad[33] * Consumos_compra_probabilidad[5555]

Se añade el stock de seguridad (columna 8) a los nuevo datos  
Se suma el stock de seguridad al nuevo punto de pedido

In [27]:
Consumos_compra_probabilidad=Consumos_compra_probabilidad.merge(s, on=0) 

In [28]:
Consumos_compra_probabilidad[66]=np.ceil(Consumos_compra_probabilidad[6] + Consumos_compra_probabilidad[8])

Los puntos de pedido calculados con media probabilistica para los artículos 1274, 1620 y 2033

In [29]:
Consumos_compra_probabilidad[Consumos_compra_probabilidad[0].isin([1274,1620,2033])].rename(columns={0:'PN',2:'SM',3:'ΣCi', 33: 'C',5555: 'Tp',8: 's', 66: 'Rp'}).drop([5,6], axis=1)

,PN,SM,ΣCi,C,Tp,s,Rp
25,1274,432.0,1570,4.313187,8.333333,60.384615,97.0
35,1620,42.0,150,0.412088,5.000000,3.296703,6.0
41,2033,1.0,1,0.002747,7.500000,0.024725,1.0


Se cuantifica el posible ahorro de costes, al multiplicar el precio de costes por la diferencia entre el stock minimo fijado por la empresa y el nuevo punto de pedido, esto es el capital invertido en el exceso de stock  
Se eliminan las filas duplicadas

In [30]:
Consumos_compra_probabilidad[7]= (Consumos_compra_probabilidad[2] - Consumos_compra_probabilidad[66]) * Consumos_compra_probabilidad[5]

In [31]:
Consumos_compra_probabilidad=Consumos_compra_probabilidad.drop_duplicates(subset=None,keep='first',inplace=False)

 Suma del valor del exceso de stock

In [32]:
np.sum(Consumos_compra_probabilidad[7])

576093.49

Se selecciona una muestra  
El punto de pedido y coste de capital invertido para Rp (ejemplo)

In [33]:
MuestraRp=Consumos_compra_probabilidad[Consumos_compra_probabilidad[0].isin([1274, 1620, 2033])]
MuestraRp=MuestraRp.rename(columns={0:"PN", 2:"SM",8: 's', 3:"ConsumoTotal",33:"ConsumoMedioDiario", 5:"PrecioMedio", 6:"Rp",66:"Rp con s", 7:"CI Rp", 5555:"TiempoMedio p"})
MuestraRp 

,PN,SM,ConsumoTotal,ConsumoMedioDiario,PrecioMedio,TiempoMedio p,Rp,s,Rp con s,CI Rp
25,1274,432.0,1570,4.313187,17.57,8.333333,35.943223,60.384615,97.0,5885.95
35,1620,42.0,150,0.412088,112.00,5.000000,2.060440,3.296703,6.0,4032.00
41,2033,1.0,1,0.002747,21.20,7.500000,0.020604,0.024725,1.0,0.00


Resultado de aplicar el punto de pedido con media probabilistica para todos los datos y su correspondiente exceso de capital invertido para cada PN (cabecera)

In [34]:
Consumos_compra_probabilidad.rename(columns={0:"PN", 2:"SM",8: 's', 3:"ConsumoTotal",33:"ConsumoMedioDiario", 5:"PrecioMedio", 6:"Rp",66:"Rp con s", 7:"CI Rp", 5555:"TiempoMedio p"}).head(5)

,PN,SM,ConsumoTotal,ConsumoMedioDiario,PrecioMedio,TiempoMedio p,Rp,s,Rp con s,CI Rp
0,1,2.0,3,0.008242,178.56,14.000000,0.115385,0.115385,1.0,178.56
1,10,1.0,9,0.024725,83.10,11.500000,0.284341,0.370879,1.0,0.00
2,36,10.0,70,0.192308,44.88,9.400000,1.807692,2.884615,5.0,224.40
3,89,4.0,44,0.120879,310.00,6.222222,0.752137,2.175824,3.0,310.00
4,92,4.0,28,0.076923,830.00,3.333333,0.256410,0.461538,1.0,2490.00


# Tercera parte


Se crea una fucion para pasa los datos de DataFrame a array de Numpy
Obtenemos los identificadores de fila y columna
se crea una matriz con el tamaño adecuado. La matriz tiene tantas filas como el mayor
identificador de fila más uno y tantas columnas como el mayor identificador de columna más uno. 
Después rellenamos la matriz. Las celdas a las que no se asigne valor quedarán a 0.

In [35]:
def df_to_numpy(dataFrame):
    rowNames=np.array(dataFrame.index)
    colNames=np.array(dataFrame.columns)
    theMatrix=np.zeros((np.max(rowNames)+1,np.max(colNames)+1))
    for curRow in rowNames:
        theMatrix[curRow,colNames]=np.array(dataFrame.loc[[curRow]])
    return theMatrix

Se establece longitud de los datos   
Posteriomente se cruzan los datos de consumo con los días del año para todos los productos

In [36]:
NUM_PRODUCTS=len(Consumos_compra) 

In [37]:
Consumo_dia_cros=Consumos.groupby([0,1])[3].sum().unstack().fillna(0) 

Se crea un DataFrame sólo con los precios  
Los precios son extraidos de los datos de los apartados anteriores

In [38]:
Precios=pd.DataFrame(Consumos_compra_probabilidad.set_index([0])[5]) 

Se convierten los datos de probabilidad, consumo y precio a array de numpy a traves de la funcion definida anteriormente

In [39]:
timeProb=df_to_numpy(probabilidad_llegada)  
theConsumption=df_to_numpy(Consumo_dia_cros)
thePrices=df_to_numpy(pd.DataFrame(Precios))

 Se extrae la columna con el precio medio y se borra el resto.

In [40]:
thePrices=np.delete(thePrices,[0,1,2,3,4],1).flatten()

La siguiente funcion permite poder extraer aquellos productos con datos relevantes ordenando los productos en funcion de su relevancia, meintras más observaciones, más relevante

In [41]:
def get_interesting_products(theConsumption,timeProb,thePrices,numProducts):
    minShape=min(timeProb.shape[0],theConsumption.shape[0],thePrices.shape[0])
    sumConsumption=(np.sum(timeProb[:minShape,:],axis=1)>0)*np.sum(theConsumption[:minShape,:]!=0,axis=1) #
    theIndices=np.argsort(sumConsumption)[::-1]
    return theIndices[:numProducts]

Productos interesantes  
En este caso son todos los productos que tengan datos de consumos y compras y un valor de stock mínimo (SM) fijado por la empresa  
Posteriormente, fijamos semilla para poder reproducir los resultados

In [42]:
myProducts=get_interesting_products(theConsumption,timeProb,thePrices,NUM_PRODUCTS) 

In [43]:
random= np.random.seed(0)

Se define el punto de pedido dinámico  
Para todo el periodo temporal  
Si el tiempo es un posible período de entrega hacemos el cálculo. 
Se suma el consumo para para cada tiempo  
Se multiplica la probabilidad por el consumo por el consumo acumulado de cada tiempo  
Al resultado se le añade el stock de seguridad, con probabilidad de rotura de stock del 5%  
Se va actualizando el punto de pedido en cada interación  
Si no lo es, la probabilitat es 0 por lo que no hace falta sumar nada.  
El resultado final se redondea a la unidad superior  

In [44]:
random 
def purchase_point(idProd,curTime,timeProb,theConsumption):
    Snew=0
    for t in range(curTime +1):
        if t<timeProb.shape[1]:
            innerSum=np.sum(theConsumption[idProd,max(0,curTime-t+1):curTime + 1])
            Snew=Snew+timeProb[idProd,min(t,timeProb.shape[1]-1)]*innerSum
            innerSumStock=np.sum(theConsumption[idProd,max(0,curTime-(int(t*0.95))+1):curTime + 1])
            Snew=Snew+timeProb[idProd,min(int(t*0.95),timeProb.shape[1]-1)]*innerSumStock
        
    return  np.ceil(Snew)

Obtenemos los indentificadores de los productos interesantes  
Creamos una matriz de ceros donde almacenar los resultados. 
El número de filases el número de productos seleccionados y el número de columnas es el número de períodos de tiempo existentes 
Para cada producto interesante  
Para todos los períodos de tiempo  
Calculamos el punto de pedido y lo almacenamos en la matriz de resultados

In [45]:
random
theProducts=get_interesting_products(theConsumption,timeProb,thePrices,NUM_PRODUCTS) 
theResults=np.zeros((NUM_PRODUCTS,theConsumption.shape[1]))
for iProduct,curProduct in enumerate(theProducts):
    for curTime in range(theConsumption.shape[1]):
        theResults[iProduct,curTime]=purchase_point(curProduct,curTime,timeProb,theConsumption)

 Se renombra el resultado de la funcion  para su posterior uso

In [46]:
purchase_point_theResults=theResults

Como hay tantos resultados como días de año, se hace la media para tener uno orientativo

In [47]:
SnewP_Pedido=np.ceil(np.mean(theResults, axis=1))

El reultado a 31/12, el último resultado del punto de pedido dinámico  
Se unifican los resultados con el nombre del producto  
La primera columna es el nombre del producto, la segunda es la media del punto de pedido y la ultima es último valor
del punto de pedido, esto es el punto de pedido a 31/12

In [48]:
SnewP_PedUlti=( theResults[:,-1]) 

In [49]:
SnewP_Pedido=np.c_[myProducts,SnewP_Pedido,SnewP_PedUlti] 

Guardamos los resultados en un data frame

In [50]:
Rd_df=pd.DataFrame(np.c_[myProducts,theResults]) 

Resultados obtenidos para Rd para diferentes t.

In [51]:
Rd_df[Rd_df[0].isin([1274,1620,2033])].loc[:,[34,60,90,333,364]] 


,34,60,90,333,364
15,16.0,384.0,0.0,84.0,58.0
201,15.0,0.0,63.0,4.0,0.0
2230,2.0,0.0,0.0,0.0,0.0


Guardamos los datos en un DataFrame  
Remplazamos los ceros por unos ya que el punto de pedido no puede ser inferior a la unidad, segun la politica de la empresa, para evitar la rotura de stock

In [52]:
MuestraRd=pd.DataFrame(SnewP_Pedido, columns=['PN', 'Rd','Rd 31-12']).replace({'Rd 31-12':{0:1}})

Se junta diferentes datos y se crean las columnas de capital invertido/ahorro para la última metodología  
Posteriormente se escogen los productos del ejemplo

In [53]:
MuestraRd=Consumos_compra_probabilidad.loc[:,[0,2,3,5,66,7]].rename(columns={0:'PN'}).merge(MuestraRd, on='PN')
MuestraRd[77]=MuestraRd[5]*(MuestraRd[2] - MuestraRd['Rd'])
MuestraRd[777]=MuestraRd[5]*(MuestraRd[2] - MuestraRd['Rd 31-12'])

Puntos de pedidos obtenidos con las diferentes metodologías,   
SM es el fijado por la empresa  
Rp es el obtenido con medias probabilistica  
Rd es la media del punto de pedido dinámico  

In [54]:
MuestraRd[MuestraRd['PN'].isin([1274,1620,2033])].loc[:,['PN',2,66,'Rd']].rename(columns={2:'SM', 66: 'Rp'})

,PN,SM,Rp,Rd
25,1274,432.0,97.0,69.0
35,1620,42.0,6.0,5.0
41,2033,1.0,1.0,1.0


Capital invertido en SM en relación a Rp y la media de Rd.

In [55]:
MuestraRd[MuestraRd['PN'].isin([1274,1620,2033])].rename(columns={2:'SM', 66: 'Rp', 5:'P', 7:'CIp', 77:'CId'}).loc[:,['PN','SM','P', 'Rp','CIp','Rd','CId']]

,PN,SM,P,Rp,CIp,Rd,CId
25,1274,432.0,17.57,97.0,5885.95,69.0,6377.91
35,1620,42.0,112.00,6.0,4032.00,5.0,4144.00
41,2033,1.0,21.20,1.0,0.00,1.0,0.00


Capital invertido en SM en relación a Rp y al último valor de Rd (31/12) .

In [56]:
MuestraRd[MuestraRd['PN'].isin([1274,1620,2033])].rename(columns={2:'SM', 66: 'Rp', 5:'P', 7:'CIp', 777:'CId a 31/12'}).loc[:,['PN','SM','P', 'Rp','CIp','Rd 31-12','CId a 31/12']]

,PN,SM,P,Rp,CIp,Rd 31-12,CId a 31/12
25,1274,432.0,17.57,97.0,5885.95,58.0,6571.18
35,1620,42.0,112.00,6.0,4032.00,1.0,4592.00
41,2033,1.0,21.20,1.0,0.00,1.0,0.00


Capital totales invertido  en el grupo B.

In [57]:
np.sum((MuestraRd[((MuestraRd[2]>1) & (MuestraRd[3]<50))].loc[:,[7,77,777]].rename(columns={7:'CIp',77: 'CId',777: 'CId 31/12'})), axis=0)

CIp          471263.08
CId          490343.20
CId 31/12    486309.74
dtype: float64

Capital invertido en el grupo A.

In [58]:
np.sum((MuestraRd[((MuestraRd[2]>1) & (MuestraRd[3]>=50))].loc[:,[7,77,777]].rename(columns={7:'CIp',77: 'CId',777: 'CId 31/12'})), axis=0)

CIp          107635.07
CId          127101.58
CId 31/12    138589.32
dtype: float64

Capital invertido en el grupo A y B.

In [59]:
np.sum((MuestraRd.loc[:,[7,77,777]].rename(columns={7:'CIp',77: 'CId',777: 'CId 31/12'})), axis=0)

CIp          576093.49
CId          616572.21
CId 31/12    606026.51
dtype: float64

# Simulación

En la simulacion están todos los datos, en el documento están exluidos los del primer grupo (con un punto de pedido igual a una unidad).  
Se exluyen en el documento porque, la política de la empresa establece que lo mínimo que debe haber en stock es una unidad, con independencia del coste

Se defini la función de costes  
Se fija:  
El precio  
El porcentaje del precio total del pedido.  
El porcentaje del precio total como coste estimado del almacenamiento.  
Qopt es la cantidad a pedir  
El costInEuros es coste total de realizar un pedido y almacenarlo

In [60]:

def coste(idProduct,meanStock,orderCount):      
    productPrice= thePrices[idProduct] 
    orderPct=0.03                 
    manCost=0.17                   
    Qopt=np.sqrt((2 * np.sum(theConsumption[idProduct])* ( thePrices[idProduct]* 0.03)) / (thePrices[idProduct] * 0.17))
    costInEuros=meanStock*productPrice*(1+manCost) + orderCount*(productPrice*orderPct)*Qopt  
    return costInEuros

### Punto de pedido dinámico

Se comienza con el punto de pedido dinámico en la simulación  
Se contrasta el punto de pedido dinámico y se le aplica la función de costes

Inicializamos el stock a 0  
Inicializamos la lista donde guardaremos todos los stocks   
Inicializamos el contador de pedidos para saber cuántos pedidos se han hecho.  
Usaremos esta variable de este modo: si vale -1 es que no hay pedido; si vale 0
es que ha llegado un pedido y si vale >0 es el número de días que faltan para
que llegue el pedido.  
theOrder representa el número de unidades que se han pedido. Sólo es válido
si theOrder no vale -1. 
Para cada instante de tiempo :
Restamos el consumo al stock.  
 Si no hay ningún pedido, calculamos el stock mínimo. Si hay
 algún pedido hecho pero no ha llegado, el stock seguirá por debajo del mínimo
 por lo que lanzaríamos continuamente nuevos pedidos.
 se iguala el stock mínimo al de punto de pedido  
 Si el stock está por debajo del mínimo (y no hay pedido en curso) lanzamos pedido.
 se genera el tiempo de entrega alaeatorio.
 La cantidad de productos que se pide es la optima .  
 Sumamos uno al contador de pedidos  
 Si ya había un pedido en curso:  
 Indicamos que falta un día menos para que llegue
 Comprobamos si ha llegado el pedido. El pedido llega si theOrder==0 o -1.
 el "-1" en este caso sólo se producirá si theOrder valía 0 antes de la
 resta y eso sólo es posible si el tiempo de entrega aleatoria ha dado 0, cosa
 que sólo pasa en productos con entrega inmediata.  
 Si llega un pedido: Aumentamos el stock y lo indicamos.
 Guardamos el stock tras cada iteración  
  

In [61]:
np.random.seed(0)
def get_random_order(theValues,theProbabilities):  
                    newProbabilities=theProbabilities.copy()
                    newProbabilities/=np.sum(newProbabilities)
                    return np.random.choice(theValues,p=newProbabilities)

def simulacion(idProduct,theConsumption,timeProb):
    theStock= 0
    stockHistory=[]
    orderCount=0
   
    theOrder = -1
    
    quantityOrder=-1
    
    sumQtyOrder=0
    
    for curTime in range(theConsumption.shape[1]):
        
        theStock=theStock -theConsumption[idProduct,curTime]
       
        if theOrder==-1:
            minStock = purchase_point(idProduct,curTime,timeProb,theConsumption)
            if theStock<=minStock:
                
                theOrder=get_random_order(range(timeProb.shape[1]),timeProb[idProduct,:]) 
                quantityOrder=np.sqrt((2 * np.sum(theConsumption[idProduct])* (thePrices[idProduct]* 0.03)) / (thePrices[idProduct] * 0.17)) 
                sumQtyOrder=sumQtyOrder+quantityOrder
                orderCount=orderCount+1

        
        else:
            theOrder=theOrder-1
            if theOrder==0 or theOrder==-1 and theStock<=minStock:
                theStock=theStock+quantityOrder
                theOrder=-1
                quantityOrder=-1
        stockHistory.append(theStock)   
        
    return stockHistory,orderCount,sumQtyOrder

En la siguiente parte de la simulación:  
Comenzamos con las variables que nos interesan
Para cada producto:  
Aplicamos la función de simulacion del punto de pedido que se esta probando  
Guardamos el stock historico, el número de veces que se ha realizado la compra y la cantidad comprada  
Igualamos el stock inicial al mínimo registrado para parir de ese punto  
Se suma el historico al inicial como punto de partida  
Posteriormente se aplica la función de costes  


In [62]:
np.random.seed(0) 

initialStockSnew=[] 
stockHistoryd=[]
allDatad=[]
for curProduct in theProducts: 
    curData=[]
         
    stockHistory,orderCount,sumQtyOrder= simulacion(curProduct,theConsumption,timeProb)     
        
    initialStock= -np.min(stockHistory)
            
    if initialStock < 0:
        initialStock = 0
        print("Revisa inicialstock", K, curProduct)
    stockHistory =stockHistory + initialStock
       
    curData.append(coste(curProduct,np.mean(stockHistory),orderCount))

    allDatad.append(curData)
    initialStockSnew.append(initialStock) 
    stockHistoryd.append(stockHistory)

allDatad =np.array(allDatad)
    

In [63]:
sum(allDatad)

array([1954685.3])

### punto de pedido con media probabilistica

Se repite el proceso anterior pero probando el punto de pedido obtenido con las probabilidades

In [64]:
np.random.seed(0)
def get_random_order(theValues,theProbabilities): 
                    newProbabilities=theProbabilities.copy()
                    newProbabilities/=np.sum(newProbabilities)
                    return np.random.choice(theValues,p=newProbabilities)

def simulacion(idProduct,theConsumption,timeProb):
    theStock= 0
    stockHistory=[]
    orderCount=0
    theOrder = -1 
    quantityOrder=-1
    
    sumQtyOrder=0
    
    for curTime in range(theConsumption.shape[1]):
        
        theStock=theStock -theConsumption[idProduct,curTime]
        if theOrder==-1:
            minStock = np.array(Consumos_compra_probabilidad[Consumos_compra_probabilidad[0]==idProduct][66])  
            if theStock<=minStock:
                
                theOrder=get_random_order(range(timeProb.shape[1]),timeProb[idProduct,:])
                quantityOrder=np.sqrt((2 * np.sum(theConsumption[idProduct])* (thePrices[idProduct]* 0.03)) / (thePrices[idProduct] * 0.17)) 
                sumQtyOrder=sumQtyOrder+quantityOrder
                orderCount=orderCount+1

        else:
            theOrder=theOrder-1
            if theOrder==0 or theOrder==-1 and theStock<=minStock:
                theStock=theStock+quantityOrder
                theOrder=-1
                quantityOrder=-1
        stockHistory.append(theStock)
        
        
    return stockHistory,orderCount,sumQtyOrder

In [65]:
np.random.seed(0)

initialStockSnew1=[]
stockHistoryp=[]
allDatap=[]
for curProduct in theProducts:
    curData=[]
    
    stockHistory,orderCount,sumQtyOrder= simulacion(curProduct,theConsumption,timeProb)  #
                
        
    initialStock= -np.min(stockHistory)
        
    if initialStock < 0:
        initialStock = 0
        print("Revisa inicialstock", K, curProduct)
    stockHistory =stockHistory + initialStock
       
    curData.append(coste(curProduct,np.mean(stockHistory),orderCount))
     
    allDatap.append(curData)
    initialStockSnew1.append(initialStock) 
    stockHistoryp.append(stockHistory)
    
allDatap =np.array(allDatap)

In [66]:
sum(allDatap)

array([2064214.9])

### punto de pedido fijado por la empresa (SM)

Se repite el mismo proceso anterior probando SM

In [67]:
np.random.seed(0)
def get_random_order(theValues,theProbabilities):
                    newProbabilities=theProbabilities.copy()
                    newProbabilities/=np.sum(newProbabilities)
                    return np.random.choice(theValues,p=newProbabilities)

def simulacion(idProduct,theConsumption,timeProb):
    theStock= 0
    stockHistory=[]
    orderCount=0
    theOrder = -1 
    quantityOrder=-1
    
    sumQtyOrder=0
    
    for curTime in range(theConsumption.shape[1]):
        
        theStock=theStock -theConsumption[idProduct,curTime]
        if theOrder==-1:
            minStock = np.array(Consumos_compra_probabilidad[Consumos_compra_probabilidad[0]==idProduct][2])  
            if theStock<=minStock:
                
                
                theOrder=get_random_order(range(timeProb.shape[1]),timeProb[idProduct,:])
                
                quantityOrder= np.sqrt((2 * np.sum(theConsumption[idProduct])* (0 + thePrices[idProduct]* 0.03) / thePrices[idProduct] * 0.17))
                
                sumQtyOrder=sumQtyOrder+quantityOrder
               
                orderCount=orderCount+1

        else:
            theOrder=theOrder-1
            if theOrder==0 or theOrder==-1 and theStock<=minStock:
                theStock=theStock+quantityOrder
                theOrder=-1
                quantityOrder=-1
        stockHistory.append(theStock)
        
        
    return stockHistory,orderCount,sumQtyOrder

In [68]:
np.random.seed(0)

initialStockSnew1=[]
stockHistorySnew=[]
allDataSM=[]
for curProduct in theProducts:
    curData=[]
            
    stockHistory,orderCount,sumQtyOrder= simulacion(curProduct,theConsumption,timeProb)  #
                
        
    initialStock= -np.min(stockHistory)
        
        
    if initialStock < 0:
        initialStock = 0
        print("Revisa inicialstock", K, curProduct)
    stockHistory =stockHistory + initialStock
       
    curData.append(coste(curProduct,np.mean(stockHistory),orderCount))

    allDataSM.append(curData)
    initialStockSnew1.append(initialStock) 
    stockHistorySnew.append(stockHistory)
    
allDataSM =np.array(allDataSM)

In [69]:
sum(allDataSM)

array([2446825.8])

Se guardan los resultados de la función de costes  
Se identifican los productos con SM =1, porque serán exluidos como ya se ha comentado anteriormente.  
'pn_mayor_1' son los productos del grupo B  
'pn_mayor_1_mayor' son los productos del grupo A

In [70]:
allda0=pd.DataFrame(np.c_[myProducts,allDatad,allDatap,allDataSM]).set_index(myProducts) 

In [71]:
pn_1=Consumos_compra_probabilidad[((Consumos_compra_probabilidad[2]==1 & (Consumos_compra_probabilidad[3]<50)))] 

In [72]:
pn_mayor_1=Consumos_compra_probabilidad[((Consumos_compra_probabilidad[2]>1) & (Consumos_compra_probabilidad[3]<50))]

In [73]:
pn_mayor_1_mayor=Consumos_compra_probabilidad[((Consumos_compra_probabilidad[2]>1) & (Consumos_compra_probabilidad[3]>=50))]

Se suma de los costes de compra y almacenamiento para el grupo B.

In [74]:
np.sum(np.round(allda0[allda0[0].isin(pn_mayor_1[0])])).rename(index={1: 'Costes Rd', 2:'Costes Rp', 3: 'Costes SM'})

0            37715428.0
Costes Rd      806808.0
Costes Rp      813388.0
Costes SM     1056656.0
dtype: float64

Se suma de los costes de compra y almacenamiento para el grupo A.

In [75]:
np.sum(np.round(allda0[allda0[0].isin(pn_mayor_1_mayor[0])])).rename(index={1: 'Costes Rd', 2:'Costes Rp', 3: 'Costes SM'})

0            7364456.0
Costes Rd     448292.0
Costes Rp     452883.0
Costes SM     551805.0
dtype: float64

Costes de compra y almacenamiento  para algunos productos del grupo B.

In [76]:
np.round(allda0[allda0[0].isin([27929,29806,52794,34102,38080,3741,27930,43709,43333,48267,30973,34611,31352,15500,9693])]).rename(columns={0:'PN', 1: 'Costes Rd', 2: 'Costes Rp', 3: 'Costes SM'})

,PN,Costes Rd,Costes Rp,Costes SM
43709,43709.0,1686.0,1841.0,1870.0
9693,9693.0,650.0,615.0,985.0
15500,15500.0,3222.0,1974.0,4242.0
30973,30973.0,9646.0,7506.0,12149.0
29806,29806.0,98.0,98.0,88.0
31352,31352.0,1756.0,1898.0,2018.0
34102,34102.0,252.0,293.0,550.0
27929,27929.0,73.0,73.0,67.0
27930,27930.0,284.0,331.0,262.0
3741,3741.0,640.0,660.0,768.0


Costes de compra y almacenamiento para algunos productos del grupo A.

In [77]:
np.round(allda0[allda0[0].isin([6364,7318,703,16021,19020,25387,26107,26453,20906,10813,36445,38408,44308,42287])]).rename(columns={0:'PN', 1: 'Costes Rd', 2: 'Costes Rp', 3: 'Costes SM'})

,PN,Costes Rd,Costes Rp,Costes SM
703,703.0,1539.0,1536.0,1958.0
10813,10813.0,979.0,861.0,1877.0
26453,26453.0,573.0,602.0,1288.0
25387,25387.0,11054.0,10051.0,19100.0
36445,36445.0,106.0,113.0,159.0
6364,6364.0,4081.0,4569.0,6970.0
42287,42287.0,1020.0,1006.0,1697.0
7318,7318.0,191.0,204.0,549.0
19020,19020.0,373.0,405.0,656.0
44308,44308.0,108.0,109.0,199.0


Resultados para algunos productos del grupo B (Punto de pedido y exceso de stock invertido respecto a SM).

In [78]:
MuestraRd[MuestraRd['PN'].isin([27929,29806,34102,38080,3741,27930,43709,43333,48267,30973,34611,31352,15500,9693])].rename(columns={2:'SM', 3:'ΣC',66: 'Rp', 5:'P', 7:'CIp',77:'CId', 777:'CId a 31/12'})

,PN,SM,ΣC,P,Rp,CIp,Rd,Rd 31-12,CId,CId a 31/12
61,3741,4.0,12,55.80,1.0,167.40,1.0,1.0,167.40,167.40
167,9693,2.0,20,141.26,2.0,0.00,2.0,1.0,0.00,141.26
884,30973,10.0,11,1543.00,1.0,13887.00,1.0,1.0,13887.00,13887.00
895,15500,2.0,14,727.14,1.0,727.14,1.0,2.0,727.14,0.00
919,29806,6.0,29,5.05,2.0,20.20,2.0,1.0,20.20,25.25
1044,34102,2.0,6,121.70,1.0,121.70,1.0,1.0,121.70,121.70
1062,31352,2.0,5,690.60,1.0,690.60,1.0,1.0,690.60,690.60
1226,34611,4.0,4,2345.00,1.0,7035.00,1.0,1.0,7035.00,7035.00
1260,38080,4.0,2,649.29,1.0,1947.87,1.0,1.0,1947.87,1947.87
1545,27929,20.0,48,1.58,1.0,30.02,1.0,1.0,30.02,30.02


Resultados para algunos productos del grupo A (Punto de pedido y exceso de stock invertido respecto a SM).

In [79]:
MuestraRd[MuestraRd['PN'].isin([6364,7318,703,16021,19020,25387,26107,26453,20906,10813,36445,38408,44308,42287])].rename(columns={2:'SM', 3:'ΣC',66: 'Rp', 5:'P', 7:'CIp',77:'CId', 777:'CId a 31/12'})

,PN,SM,ΣC,P,Rp,CIp,Rd,Rd 31-12,CId,CId a 31/12
105,6364,50.0,132,93.17,8.0,3913.14,6.0,1.0,4099.48,4565.33
120,7318,12.0,157,7.02,17.0,-35.10,17.0,1.0,-35.10,77.22
147,703,20.0,219,20.75,6.0,290.50,5.0,1.0,311.25,394.25
277,16021,4.0,70,36.40,6.0,-72.80,6.0,1.0,-72.80,109.20
334,19020,100.0,365,2.27,19.0,183.87,10.0,150.0,204.30,-113.50
536,25387,20.0,78,491.00,5.0,7365.00,4.0,7.0,7856.00,6383.00
575,26107,30.0,193,41.90,5.0,1047.50,3.0,1.0,1131.30,1215.10
592,26453,30.0,288,8.45,12.0,152.10,10.0,36.0,169.00,-50.70
781,20906,46.0,160,29.35,12.0,997.90,12.0,1.0,997.90,1320.75
1085,10813,6.0,58,76.46,6.0,0.00,6.0,2.0,0.00,305.84
